In [1]:
import pickle
import time
import os
import datetime
import numpy as np
import pandas as pd

cur_dir = os.getcwd()
main_dir = os.path.dirname(os.path.dirname(cur_dir))

In [2]:
test_date = "08_02_2022"
test_folder = "Day2_Training1_B"
file_names = {"eds":"DateTimed_WTRUN2_day2_training1_B_EDS", 
              "uav":"WTRUN2_day2_training1_B_UAV",
              "rtdstr":"compensated_normalized_WTRUN2_day2_training1_B_2022-08-02_17-25-43-99_rtd-str",
              "pzt":"WTRUN2_day2_training1_B_2022-08-02_17-25-43-99_pzt_ch3F00FF00_0.05V"}

In [3]:
#Bring in the EDS, UAV, and RTD-STR (IMGenie) data as Pandas Dataframes
test_dfs = dict()
data_dir = os.path.join(main_dir, test_date+"_Tests", "testdata", test_folder)

for sensor_type in ("eds", "uav", "rtdstr"):
  test_csv = os.path.join(data_dir, file_names[sensor_type]+".csv")
  test_df = pd.read_csv(test_csv, header=0)
  test_dfs[sensor_type] = test_df

In [4]:
# First, add "Label" column to indicate flight condition where flag==0b10011 (capturing data) - ONLY FOR TRAINING DATA
eds_df = test_dfs["eds"]
eds_df.insert(1, "Label", None)
airspeeds = [7, 8.3, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
aoas = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

conds = list()
for airspeed in airspeeds:
  for aoa in aoas:
    conds.append (f"{airspeed}m/s_{aoa}deg")

cond_ix = -1
prev_flag = 0
for row_ix in range(eds_df.shape[0]):
  try: #Here try-except enforces labeling only for training part of the run.
    if row_ix % 20000 == 0:
      print (row_ix)
    row = eds_df.iloc[row_ix]
    if row["Flag"] == "0b10011":
      if prev_flag == 0:
        cond_ix += 1
        prev_flag = 1
      eds_df.loc[row_ix, "Label"] = conds[cond_ix]
    else:
      prev_flag = 0
  except:
    pass

0
20000
40000
60000
80000
100000
120000
140000


In [5]:
#Insert DateTime objects to the dataframes.

#Insert for EDS data:
eds_times = test_dfs['eds']["DateTime Str"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f"))
test_dfs['eds'].insert(1, "eds_DateTime Obj", eds_times)

#Insert for RTD-STR data:
month = int (test_date[0:2])
day = int (test_date[3:5])
year = int (test_date[6:])
rtdstr_times = test_dfs['rtdstr']["Date/Time"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d_%H-%M-%S-%f"))
rtdstr_times = rtdstr_times.apply(lambda x: datetime.datetime(year, month, day, x.hour, x.minute, x.second, x.microsecond)) #We do this because we haven't recorded date in the original data.
test_dfs['rtdstr'].insert(1, "rtdstr_DateTime Obj", rtdstr_times)

#Insert for UAV data:
#First define a function to separate full seconds into hours, minutes, seconds, and milliseconds
def seperate_seconds(seconds):
  #This function assumes inserted seconds is shorter than 24 hours = 86,400 seconds
  onlyseconds = int(seconds.split(".")[0])
  hours = int (onlyseconds/(60*60))
  minutes = int (onlyseconds/(60)%60)
  remseconds = int (onlyseconds - hours*60*60 - minutes*60)
  milliseconds = seconds.split(".")[1][0:6]
  return hours, minutes, remseconds, milliseconds

seconds = test_dfs['uav']["STATIC_PRESSURE_TIME"].astype(str)
time_sr = seconds.apply(seperate_seconds)
uav_times = time_sr.astype(str).apply(lambda x: datetime.datetime.strptime(x, "(%H, %M, %S, '%f')"))
test_dfs['uav'].insert(1, "uav_DateTime Obj", uav_times)

In [6]:
# Adjust UAV's uav_DateTime Obj by first finding the 10 degree sync point on the UAV and EDS.
# And correcting it using EDS's.
observe_column_name = {"eds":"Inclination (deg)", "uav":"Pitch (deg)"}
aoajump_ixs = dict()
for data_stream in ("uav", "eds"):
  #Find the indices of eds and uav streams where Inclination (deg) and Pitch (deg) increases by >4 degrees in 5 seconds.
  
  #Calculate the average time delta between each row.
  time_delta = (test_dfs[data_stream][f"{data_stream}_DateTime Obj"][200] - test_dfs[data_stream][f"{data_stream}_DateTime Obj"][100])/100
  
  #Calculate diff for 5 seconds
  num_rows = int(5/time_delta.total_seconds())
  angle_change = test_dfs[data_stream][observe_column_name[data_stream]].diff(periods=num_rows)

  #Extract the row ID where we see the first >4 degree change in 5 seconds (+10 degree calibration procedure).
  aoajump_ix = angle_change[angle_change>4].index[0]
  aoajump_ixs[data_stream] = aoajump_ix

eds_aoajump_timestamp = test_dfs["eds"]["eds_DateTime Obj"][aoajump_ixs["eds"]]
uav_aoajump_timestamp = test_dfs["uav"]["uav_DateTime Obj"][aoajump_ixs["uav"]]
uav_timedelta = eds_aoajump_timestamp - uav_aoajump_timestamp

test_dfs['uav']["uav_DateTime Obj"] += uav_timedelta


In [7]:
training_end_ix = eds_df.loc[eds_df["Label"] == "20m/s_16deg"].iloc[-1].name
training_end_time = eds_df.loc[training_end_ix]["eds_DateTime Obj"]
training_end_time += datetime.timedelta(0,120,0) #Adding two minutes offset because we wait a resonable amount between training and dynamic tests.

In [8]:
training_end_time

Timestamp('2022-08-02 20:28:56.681000')

In [9]:
#Determine the cutoff indices of data except for PZT
cutoff_indices = dict()
training_end_times = dict()
for data_source in ("uav", "rtdstr", "eds"):
  timedelta = datetime.timedelta(1)
  ix = 0
  while abs(training_end_time - test_dfs[data_source].iloc[ix][f"{data_source}_DateTime Obj"]) < timedelta:
    timedelta = training_end_time - test_dfs[data_source].iloc[ix][f"{data_source}_DateTime Obj"]
    ix += 10 #take big steps to step up the counter for speed as we don't have to be exact here.
  cutoff_indices[data_source] = ix
  training_end_times[data_source] = test_dfs[data_source].iloc[ix][f"{data_source}_DateTime Obj"]

In [10]:
cutoff_indices['uav'] -= 500 #We move back UAV's cutoff index for consistency (we always assume UAV starts earlier than others)
training_end_times['uav'] = test_dfs['uav'].iloc[cutoff_indices['uav']]["uav_DateTime Obj"]

In [11]:
cutoff_indices

{'uav': 595430, 'rtdstr': 1419270, 'eds': 100430}

In [12]:
#Write the split data into new csv files (except PZT)
for data_source in ("uav", "rtdstr", "eds"):
  for data_type in ("training", "dynamic"):
    output_folder = os.path.join(main_dir, test_date+"_Tests", "testdata", test_folder+f"_{data_type}")
    if data_type == "training":
      output_csv = os.path.join(output_folder, f"{data_type}_{data_source}.csv")
      test_dfs[data_source].loc[0:cutoff_indices[data_source]].to_csv(output_csv, index=False)
    else:
      output_csv = os.path.join(output_folder, f"{data_type}_{data_source}.csv")
      test_dfs[data_source].loc[cutoff_indices[data_source]:].to_csv(output_csv, index=False)

In [13]:
# Determine the cutoff index of PZT data
start_time_rtdstr = test_dfs['eds'].iloc[0]["eds_DateTime Obj"]
end_time_rtdstr = test_dfs['eds'].iloc[cutoff_indices['eds']]["eds_DateTime Obj"]

time_elapsed_rtdstr = end_time_rtdstr - start_time_rtdstr
minutes_elapsed_rtdstr = int (time_elapsed_rtdstr.total_seconds() // 60)
pzt_csv_to_split = int(minutes_elapsed_rtdstr // 30) + 1 #Assuming PZT binaries are split per 30 minutes

seconds_into = time_elapsed_rtdstr.total_seconds() - (pzt_csv_to_split-1)*30*60
lines_into = seconds_into * 10000
cutoff_indices['pzt'] = [str(pzt_csv_to_split), str(int(lines_into))] #(Suffix of PZT .csv file, index of cutoff line)

In [14]:
cutoff_indices

{'uav': 595430, 'rtdstr': 1419270, 'eds': 100430, 'pzt': ['7', '1893950']}

In [15]:
pzt_csv = os.path.join(data_dir, file_names['pzt']+"_"+cutoff_indices['pzt'][0]+".csv")
pzt_df = pd.read_csv(pzt_csv, header=0)

training_output_csv = os.path.join(main_dir, test_date+"_Tests", "testdata", test_folder+"_training", f"training_pzt_{cutoff_indices['pzt'][0]}.csv")
pzt_df.loc[0:cutoff_indices['pzt'][1]].to_csv(training_output_csv, index=False)

dynamic_output_csv = os.path.join(main_dir, test_date+"_Tests", "testdata", test_folder+"_dynamic", f"dynamic_pzt_{cutoff_indices['pzt'][0]}.csv")
pzt_df.loc[cutoff_indices['pzt'][1]:].to_csv(dynamic_output_csv, index=False)